In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
base = Path.cwd().parents[1]
df = pd.read_csv(base / "Data_clean/caract-2005-2023.csv")

df


C:\Users\jeanl\AppData\Local\Temp\ipykernel_16296\1827805698.py:8: DtypeWarning: Columns (10,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(base / "Data_clean/caract-2005-2023.csv")


,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,date,weekday
1121571,202200000001,2022,10,19,16:15,1,2,3,1.0,3.0,26198,TEIL(vieille route du),NaN,44.559420,4.725720,26,2022-10-19,3
1121572,202200000002,2022,10,20,08:34,1,2,3,1.0,3.0,25204,Miranda,NaN,46.925810,6.346200,25,2022-10-20,4
1121573,202200000003,2022,10,20,17:15,1,2,6,1.0,2.0,22360,ROND POINT DE BREZILLET,NaN,48.493162,-2.760439,22,2022-10-20,4
1121574,202200000004,2022,10,20,18:00,1,2,3,8.0,6.0,16102,LOHMEYER (RUE),NaN,45.692652,-0.326290,16,2022-10-20,4
1121575,202200000005,2022,10,19,11:45,1,1,1,1.0,2.0,13103,ROUTE DE JEAN MOULIN-RN 538,NaN,43.675579,5.092703,13,2022-10-19,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176868,202200055298,2022,1,1,03:50,3,1,1,1.0,6.0,2B293,D71,NaN,42.310165,9.478583,2B,2022-01-01,6
1176869,202200055299,2022,1,1,07:20,3,1,1,1.0,6.0,84074,D973,NaN,43.753164,5.224476,84,2022-01-01,6
1176870,202200055300,2022,1,1,04:27,3,1,1,9.0,6.0,74001,D22,NaN,46.282532,6.732806,74,2022-01-01,6
1176871,202200055301,2022,1,1,08:40,1,1,3,1.0,3.0,81099,Chemin Toulze,NaN,43.927265,1.915637,81,2022-01-01,6


## Fusion des datasets

In [3]:
base = Path.cwd().parents[1]
df_lieux = pd.read_csv(base / "Data_clean/lieux-2005-2023.csv")
df_vehicules = pd.read_csv(base / "Data_clean/vehicules-2005-2023.csv", index_col=0)
df_usagers = pd.read_csv(base / "Data_clean/usagers-2005-2023.csv",index_col=0)
df["Num_Acc"] = df["Num_Acc"].fillna(0).astype(int)
df_lieux.head()


C:\Users\jeanl\AppData\Local\Temp\ipykernel_16296\3338191486.py:2: DtypeWarning: Columns (2,6,7,8,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lieux = pd.read_csv(base / "Data_clean/lieux-2005-2023.csv")
C:\Users\jeanl\AppData\Local\Temp\ipykernel_16296\3338191486.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicules = pd.read_csv(base / "Data_clean/vehicules-2005-2023.csv", index_col=0)
C:\Users\jeanl\AppData\Local\Temp\ipykernel_16296\3338191486.py:4: DtypeWarning: Columns (9,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_usagers = pd.read_csv(base / "Data_clean/usagers-2005-2023.csv",index_col=0)


,Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma
0,200500000001,3.0,41.0,0.0,B,2.0,2,1.0,430.0,0.0,1.0,1.0,0,63,1.0,0.0,1.0,0.0,NaN
1,200500000002,2.0,41.0,0.0,NaN,0.0,2,0.0,0.0,1.0,1.0,1.0,0,100,1.0,0.0,5.0,0.0,NaN
2,200500000003,2.0,41.0,0.0,NaN,0.0,0,0.0,0.0,1.0,1.0,1.0,0,0,2.0,0.0,5.0,0.0,NaN
3,200500000004,3.0,916.0,0.0,NaN,2.0,2,0.0,0.0,0.0,1.0,1.0,0,0,1.0,0.0,1.0,0.0,NaN
4,200500000005,3.0,110.0,0.0,NaN,2.0,2,24.0,630.0,0.0,1.0,3.0,0,59,2.0,0.0,3.0,0.0,NaN


In [4]:
df_complet = df.merge(df_lieux, on="Num_Acc", how="inner")
df_complet = df_complet.drop_duplicates(subset=['Num_Acc'], keep='first')

In [5]:
df_complet = df_complet.merge(df_vehicules, on="Num_Acc", how = "inner")


In [6]:
df_complet = df_complet.merge(df_usagers, on="Num_Acc", how = "inner")

In [7]:
pd.set_option('display.max_columns', None)
df_complet

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,date,weekday,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma,senc,catv,occutc,obs,obsm,choc,manv,num_veh_x,id_vehicule_x,motor,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh_y,id_vehicule_y,secu1,secu2,secu3,id_usager
0,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,11,CD41B,M,50.51500,2.9440,590,2005-01-12,3,3.0,41.0,0.0,B,2.0,2,1.0,430.0,0.0,1.0,1.0,0,63,1.0,0.0,1.0,0.0,NaN,0.0,7,0.0,0.0,2.0,1.0,1.0,A01,NaN,NaN,1,1,4,1,1,11,0,0,0,1976,A01,NaN,0,0,0,NaN
1,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,11,CD41B,M,50.51500,2.9440,590,2005-01-12,3,3.0,41.0,0.0,B,2.0,2,1.0,430.0,0.0,1.0,1.0,0,63,1.0,0.0,1.0,0.0,NaN,0.0,7,0.0,0.0,2.0,1.0,1.0,A01,NaN,NaN,1,1,3,2,3,11,0,0,0,1968,B02,NaN,0,0,0,NaN
2,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,11,CD41B,M,50.51500,2.9440,590,2005-01-12,3,3.0,41.0,0.0,B,2.0,2,1.0,430.0,0.0,1.0,1.0,0,63,1.0,0.0,1.0,0.0,NaN,0.0,7,0.0,0.0,2.0,1.0,1.0,A01,NaN,NaN,2,2,1,1,0,11,0,0,0,1964,B02,NaN,0,0,0,NaN
3,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,11,CD41B,M,50.51500,2.9440,590,2005-01-12,3,3.0,41.0,0.0,B,2.0,2,1.0,430.0,0.0,1.0,1.0,0,63,1.0,0.0,1.0,0.0,NaN,0.0,7,0.0,0.0,2.0,1.0,1.0,A01,NaN,NaN,4,2,1,1,0,31,0,0,0,2004,B02,NaN,0,0,0,NaN
4,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,11,CD41B,M,50.51500,2.9440,590,2005-01-12,3,3.0,41.0,0.0,B,2.0,2,1.0,430.0,0.0,1.0,1.0,0,63,1.0,0.0,1.0,0.0,NaN,0.0,7,0.0,0.0,2.0,1.0,1.0,A01,NaN,NaN,5,2,1,1,0,11,0,0,0,1998,B02,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242669,202300054822,2023,10,20,16:30,1,2,1,6.0,3.0,69387,Boulevard Yves Farge,NaN,45.73306,4.8254,69,2023-10-20,5,4.0,YVES FARGES (BD),0.0,NaN,2.0,2,-1,-1,2.0,1.0,1.0,NaN,-1,1.0,0.0,5.0,NaN,30.0,2.0,7,NaN,0.0,2.0,7.0,22.0,A01,155 583 344,1.0,1,1,1,2,1,0,-1,-1,-1,2002,A01,155 583 344,1,-1,-1,203 720 718
5242670,202300054822,2023,10,20,16:30,1,2,1,6.0,3.0,69387,Boulevard Yves Farge,NaN,45.73306,4.8254,69,2023-10-20,5,4.0,YVES FARGES (BD),0.0,NaN,2.0,2,-1,-1,2.0,1.0,1.0,NaN,-1,1.0,0.0,5.0,NaN,30.0,2.0,7,NaN,0.0,2.0,7.0,22.0,A01,155 583 344,1.0,1,1,3,2,1,0,-1,-1,-1,1995,B01,155 583 345,2,-1,-1,203 720 719
5242671,202300054822,2023,10,20,16:30,1,2,1,6.0,3.0,69387,Boulevard Yves Farge,NaN,45.73306,4.8254,69,2023-10-20,5,4.0,YVES FARGES (BD),0.0,NaN,2.0,2,-1,-1,2.0,1.0,1.0,NaN,-1,1.0,0.0,5.0,NaN,30.0,2.0,50,NaN,14.0,2.0,1.0,1.0,B01,155 583 345,3.0,2,2,1,2,2,0,-1,-1,-1,2003,A01,155 583 344,1,-1,-1,203 720 717
5242672,202300054822,2023,10,20,16:30,1,2,1,6.0,3.0,69387,Boulevard Yves Farge,NaN,45.73306,4.8254,69,2023-10-20,5,4.0,YVES FARGES (BD),0.0,NaN,2.0,2,-1,-1,2.0,1.0,1.0,NaN,-1,1.0,0.0,5.0,NaN,30.0,2.0,50,NaN,14.0,2.0,1.0,1.0,B01,155 583 345,3.0,1,1,1,2,1,0,-1,-1,-1,2002,A01,155 583 344,1,-1,-1,203 720 718


In [8]:
df_complet.to_csv(base / "Data_clean/full.csv", index=False)